<a href="https://colab.research.google.com/github/Aroobmushtaq/Ai/blob/main/06_agents/Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai python-dotenv pydantic


In [ ]:
from openai import OpenAI

#  Paste your Gemini API key directly here
GEMINI_API_KEY = "past api key here or in secret"

# Initialize the OpenAI client with Gemini base URL
client = OpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

# Ask user input
query = input("Enter your question: ")

# Send query to Gemini model
response = client.chat.completions.create(
    model="gemini-2.0-flash",   # You can try gemini-1.5-flash if this fails
    messages=[
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": query}
    ]
)

# Print output
print("\nAssistant:", response.choices[0].message.content)


**Practice with Context**

Context is just a container (or a box) that holds the data the agent needs while it’s running.
For example, the context might store things like:

* The user’s name or ID

* Settings or preferences

* Extra data the agent needs to do its job

In [ ]:
import os
from dataclasses import dataclass
from dotenv import load_dotenv

# --- Load or set API key ---
os.environ["GEMINI_API_KEY"] = "key here"
gemini_api_key = os.getenv("GEMINI_API_KEY")

# --- Import OpenAI client ---
from openai import AsyncOpenAI

# --- Mock classes (to simulate missing ones) ---
class OpenAIChatCompletionsModel:
    def __init__(self, model, openai_client):
        self.model = model
        self.client = openai_client

class Agent:
    def __init__(self, name, instructions, model, tools):
        self.name = name
        self.instructions = instructions
        self.model = model
        self.tools = tools

class Runner:
    @staticmethod
    def run_sync(agent, query, context):
        for tool in agent.tools:
            info = tool(context)
            return type("Result", (), {"final_output": f"Query: {query}\n{info}"})

def function_tool(func):
    return func

class RunContextWrapper:
    def __init__(self, context):
        self.context = context

# --- Create AsyncOpenAI client ---
client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

# --- User class ---
@dataclass
class User:
    user_id: int

#  FIXED HERE: remove `[User]` — just use RunContextWrapper normally
@function_tool
def get_user_info(ctx: RunContextWrapper) -> str:
    """Fetches user personal information."""
    id = ctx.context.user_id
    if id == 1:
        return "User name is Ali. He is 19 years old. He is an Agentic AI Engineer who likes playing Cricket."
    elif id == 2:
        return "User name is Usman. He is 30 years old. He is a doctor who likes mountains."
    else:
        return "User not found"

# --- Create Agent ---
agent = Agent(
    name="Assistant",
    instructions="You are an expert in agentic AI.",
    model=OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=client),
    tools=[get_user_info]
)

# --- Run simulation ---
query = input("Enter the query: ")

result = Runner.run_sync(
    agent,
    query,
    context=RunContextWrapper(User(user_id=1))
)

print(result.final_output)


*Practice for output type in agents*

In [ ]:
# ------------------------- CREATE agents.py FILE -------------------------
import textwrap

code = textwrap.dedent("""
class Agent:
    def __init__(self, name, instructions, model):
        self.name = name
        self.instructions = instructions
        self.model = model

class OpenAIChatCompletionsModel:
    def __init__(self, model, openai_client):
        self.model = model
        self.client = openai_client

class Runner:
    @staticmethod
    def run_sync(agent, query):
        response = agent.model.client.chat.completions.create(
            model=agent.model.model,
            messages=[
                {"role": "system", "content": agent.instructions},
                {"role": "user", "content": query}
            ]
        )
        return type("Result", (), {"final_output": response.choices[0].message.content})
""")

with open("agents.py", "w") as f:
    f.write(code)

# ------------------------- MAIN SCRIPT -------------------------
from openai import OpenAI
from agents import Agent, OpenAIChatCompletionsModel, Runner
from pydantic import BaseModel
from typing import List

# Hard-coded Gemini API key  (replace with your actual key)
GEMINI_API_KEY = "your key here"

# Create Gemini client (sync version)
client = OpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

# Define data model for quiz
class Quiz(BaseModel):
    question: str
    options: List[str]
    correct_option: str

# Create agent
agent = Agent(
    name="Assistant",
    instructions="You are a Quiz Agent. You generate quizzes with question, 4 options, and correct answer.",
    model=OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=client),
    output_type=Quiz
)

# Run in Colab
query = input("Enter your quiz topic: ")

result = Runner.run_sync(agent, query)

print("\n--- Quiz Generated ---\n")
print(result.final_output)


**Handoffs**

Handoffs are sub‑agents the agent can delegate to. When a handoff occurs, the delegated agent receives the conversation history and takes over the conversation. This pattern enables modular, specialized agents that excel at a single task. Read more in the handoffs documentation.

In [ ]:
# ------------------------ CREATE agents.py FILE ------------------------
import textwrap

code = textwrap.dedent("""
class Agent:
    def __init__(self, name, instructions, model, tools=None, handoffs=None):
        self.name = name
        self.instructions = instructions
        self.model = model
        self.tools = tools or []
        self.handoffs = handoffs or []

class OpenAIChatCompletionsModel:
    def __init__(self, model, openai_client):
        self.model = model
        self.client = openai_client

class Runner:
    @staticmethod
    def run_sync(agent, query):
        response = agent.model.client.chat.completions.create(
            model=agent.model.model,
            messages=[
                {"role": "system", "content": agent.instructions},
                {"role": "user", "content": query}
            ]
        )
        return type("Result", (), {"final_output": response.choices[0].message.content})

def function_tool(func):
    return func
""")

with open("agents.py", "w") as f:
    f.write(code)

# ------------------------ MAIN SCRIPT ------------------------
from openai import OpenAI
from agents import Agent, OpenAIChatCompletionsModel, Runner, function_tool

#  Hardcode your Gemini API key here
GEMINI_API_KEY = "your key here"

# Create client
client = OpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

# Define tools
@function_tool
def get_refund(amount) -> str:
    print(f"Fetching refund status for {amount}...")
    return "Refund approved"

@function_tool
def book_flight(location) -> str:
    print(f"Booking flight for {location}...")
    return "Flight booked successfully."

# Define agents
booking_agent = Agent(
    name="Booking agent",
    instructions="You are a booking agent.",
    model=OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=client),
    tools=[book_flight],
)

refund_agent = Agent(
    name="Refund agent",
    instructions="You are a refund agent.",
    model=OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=client),
    tools=[get_refund],
)

triage_agent = Agent(
    name="Triage agent",
    instructions=(
        "Help the user with their questions. "
        "If they ask about booking, handoff to the booking agent. "
        "If they ask about refunds, handoff to the refund agent."
    ),
    model=OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=client),
    handoffs=[booking_agent, refund_agent],
)

# Instead of input(), just assign your query directly
query = "I want to book a flight to Karachi"

# Run
result = Runner.run_sync(triage_agent, query)

print("\\n--- Final Output ---\\n")
print(result.final_output)


**Dynamic instructions**

In [ ]:

import os
from dataclasses import dataclass
from dotenv import load_dotenv
from openai import AsyncOpenAI

# ---------------------------
# Load or set your API key
# ---------------------------
load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY") or "api key here"

# Create the OpenAI client
client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

# ---------------------------
# Define user context
# ---------------------------
@dataclass
class UserContext:
    name: str

# ---------------------------
# Define dynamic instruction function
# ---------------------------
def dynamic_instructions(user_context: UserContext) -> str:
    return f"The user's name is {user_context.name}. Help them with their questions."

# ---------------------------
# Run the query
# ---------------------------
import asyncio

async def run_query():
    user = UserContext(name="Aroob")
    query = input("Enter your query: ")

    instructions = dynamic_instructions(user)

    response = await client.chat.completions.create(
        model="gemini-2.0-flash",
        messages=[
            {"role": "system", "content": instructions},
            {"role": "user", "content": query},
        ],
    )

    print("\nFinal Output:\n")
    print(response.choices[0].message.content)

# Run async function in Colab
await run_query()


**Cloning/copying agents**

In [ ]:
!pip install openai python-dotenv nest_asyncio --quiet

import os, asyncio, nest_asyncio, warnings
from dotenv import load_dotenv
from openai import AsyncOpenAI

# ---------------------- SETUP ----------------------
nest_asyncio.apply()  # Fix async loop issue in Colab
warnings.filterwarnings("ignore", category=RuntimeWarning)  # hide async warnings
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY") or "key past here"

client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

# ---------------------- DEFINE CLASSES ----------------------
class Agent:
    def __init__(self, name, instructions, model):
        self.name = name
        self.instructions = instructions
        self.model = model

    def clone(self, **kwargs):
        params = {
            "name": self.name,
            "instructions": self.instructions,
            "model": self.model
        }
        params.update(kwargs)
        return Agent(**params)

class Runner:
    @staticmethod
    async def run(agent, query):
        """Async Gemini API call"""
        response = await client.chat.completions.create(
            model=agent.model,
            messages=[
                {"role": "system", "content": agent.instructions},
                {"role": "user", "content": query},
            ],
        )
        return response.choices[0].message.content

    @staticmethod
    def run_sync(agent, query):
        """Safe run for Colab (no warnings)"""
        loop = asyncio.get_event_loop()
        task = loop.create_task(Runner.run(agent, query))
        loop.run_until_complete(task)
        return task.result()

# ---------------------- CREATE & CLONE AGENTS ----------------------
pirate_agent = Agent(
    name="Pirate",
    instructions="Talk like a friend",
    model="gemini-2.0-flash"
)

robot_agent = pirate_agent.clone(
    name="Robot",
    instructions="Respond like a robot using mechanical and logical tone."
)

# ---------------------- RUN BOTH AGENTS ----------------------
query = input("Enter your query: ")

print("\n--- Pirate Agent Says ---")
print(Runner.run_sync(pirate_agent, query))

print("\n--- Robot Agent Says ---")
print(Runner.run_sync(robot_agent, query))


**Forcing tool use**

In [ ]:
!pip install openai python-dotenv nest_asyncio --quiet

import os, asyncio, nest_asyncio
from dotenv import load_dotenv
from openai import AsyncOpenAI

# Allow nested event loops in Colab
nest_asyncio.apply()

# ------------------ Load API key ------------------
load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY") or "YOUR_GEMINI_API_KEY_HERE"

# ------------------ Setup Gemini client ------------------
client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

# ------------------ Define a "tool" ------------------
def get_weather(city: str) -> str:
    """Fake weather tool"""
    return f"The weather in {city} is sunny "

# ------------------ Agent simulation ------------------
class Agent:
    def __init__(self, name, instructions, model, tools=None, tool_choice="auto"):
        self.name = name
        self.instructions = instructions
        self.model = model
        self.tools = tools or []
        self.tool_choice = tool_choice

class Runner:
    @staticmethod
    async def run(agent, query):
        if agent.tool_choice == "required" and agent.tools:
            # Simple way to extract city
            city = query.split()[-1]
            tool = agent.tools[0]
            return tool(city)
        else:
            response = await client.chat.completions.create(
                model=agent.model,
                messages=[
                    {"role": "system", "content": agent.instructions},
                    {"role": "user", "content": query},
                ],
            )
            return response.choices[0].message.content

# ------------------ Create agent ------------------
agent = Agent(
    name="Weather Assistant",
    instructions="You are a helpful assistant that provides weather information.",
    model="gemini-2.0-flash",
    tools=[get_weather],
    tool_choice="required"  # Force tool use
)

# ------------------ Run in Colab ------------------
query = input("Enter your query (e.g. What's the weather in Lahore?): ")

result = await Runner.run(agent, query)

print("\n--- Final Output ---")
print(result)


**Running agents**

**Synchronously Running**

In [ ]:
!pip install openai python-dotenv nest_asyncio --quiet

import os, asyncio, nest_asyncio
from dotenv import load_dotenv
from openai import AsyncOpenAI

# Allow event loop reuse (required for Colab)
nest_asyncio.apply()

# -------------------- SETUP GEMINI CLIENT --------------------
os.environ["GEMINI_API_KEY"] = "key here"  #  Replace this with your Gemini API key
load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY")

client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

# -------------------- DEFINE SIMPLE CLASSES --------------------
class OpenAIChatCompletionsModel:
    def __init__(self, model, openai_client):
        self.model = model
        self.client = openai_client

class Agent:
    def __init__(self, name, instructions, model):
        self.name = name
        self.instructions = instructions
        self.model = model

class Runner:
    @staticmethod
    def run_sync(agent, query):
        """Run synchronously (safe for Colab)."""
        async def _run():
            response = await agent.model.client.chat.completions.create(
                model=agent.model.model,
                messages=[
                    {"role": "system", "content": agent.instructions},
                    {"role": "user", "content": query},
                ],
            )
            return response.choices[0].message.content

        loop = asyncio.get_event_loop()
        return loop.run_until_complete(_run())

# -------------------- CREATE & RUN AGENT --------------------
agent = Agent(
    name="Assistant",
    instructions="You are an expert of agentic AI. Explain things simply.",
    model=OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=client),
)

query = input("Enter your question: ")
result = Runner.run_sync(agent, query)

print("\n--- Final Output ---")
print(result)


**Asynchronous Running**

In [ ]:
import asyncio

# -------------------- ASYNC RUNNER CLASS --------------------
class Runner:
    @staticmethod
    async def run(agent, query):
        """Run the agent asynchronously."""
        response = await agent.model.client.chat.completions.create(
            model=agent.model.model,
            messages=[
                {"role": "system", "content": agent.instructions},
                {"role": "user", "content": query},
            ],
        )
        return response.choices[0].message.content


# -------------------- MAIN ASYNC FUNCTION --------------------
async def main():
    agent = Agent(
        name="Assistant",
        instructions="You are an expert of agentic AI. Explain things simply.",
        model=OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=client),
    )

    query = input("Enter your question: ")
    result = await Runner.run(agent, query)

    print("\n--- Final Output (Async) ---")
    print(result)

# -------------------- RUN ASYNC FUNCTION --------------------
await main()


**Streaming Running**

In [ ]:
!pip install openai nest_asyncio -q

import nest_asyncio, asyncio
from openai import AsyncOpenAI
from openai.types.responses import ResponseTextDeltaEvent

nest_asyncio.apply()

# --- Simple custom Agent + Runner classes for Colab ---
class Agent:
    def __init__(self, name, instructions, model):
        self.name = name
        self.instructions = instructions
        self.model = model

class Runner:
    @staticmethod
    def run_streamed(agent, input_text):
        async def _stream():
            stream = await client.chat.completions.create(
                model=agent.model,
                messages=[
                    {"role": "system", "content": agent.instructions},
                    {"role": "user", "content": input_text},
                ],
                stream=True,
            )
            async for chunk in stream:
                if hasattr(chunk.choices[0].delta, "content"):
                    print(chunk.choices[0].delta.content or "", end="", flush=True)
        return _stream()

# --- Gemini setup ---
gemini_api_key = "key here"

client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

# --- Async main to run stream ---
async def main():
    agent = Agent(
        name="Assistant",
        instructions="You are an AI expert who answers clearly.",
        model="gemini-2.0-flash"
    )

    query = input("Enter your query: ")
    print("\n--- Response ---\n")
    await Runner.run_streamed(agent, query)

await main()


**Run Config**

**RunConfig** allows you to set global configurations for an agent run without modifying the agent itself.
It lets you override settings such as the model, model provider, temperature, tracing options, and safety guardrails.
Using **RunConfig** provides flexibility and control — for example, you can switch models or adjust behavior for a single run while keeping your base agent unchanged.

In [ ]:
!pip install openai nest_asyncio -q

import nest_asyncio, asyncio
from openai import AsyncOpenAI

# Allow Colab to run async functions inside cells
nest_asyncio.apply()

# ------------------ Simplified Local Agent Setup ------------------
class Agent:
    def __init__(self, name, instructions):
        self.name = name
        self.instructions = instructions

class Runner:
    @staticmethod
    def run_sync(agent, query, run_config=None):
        # run_config lets you override the model or tracing settings
        model_to_use = run_config.get("model") if run_config else "gemini-2.0-flash"

        # Since Colab can’t directly stream sync with AsyncOpenAI, we use asyncio
        async def _run():
            response = await client.chat.completions.create(
                model=model_to_use,
                messages=[
                    {"role": "system", "content": agent.instructions},
                    {"role": "user", "content": query},
                ]
            )
            return response.choices[0].message.content

        return asyncio.get_event_loop().run_until_complete(_run())

# ------------------ RunConfig (mocked for Colab) ------------------
class RunConfig(dict):
    """A simple placeholder class to simulate configuration."""
    pass

# ------------------ Gemini Setup ------------------
gemini_api_key = "key here"

client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

# ------------------ Example Usage ------------------
run_config = RunConfig({
    "model": "gemini-2.0-flash",  # same as the real RunConfig.model
    "tracing_disabled": True
})

agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant."
)

query = input("Enter your query: ")
result = Runner.run_sync(agent, query, run_config)
print("\n--- Final Output ---\n", result)


**Chat Threads**

In [ ]:
!pip install openai nest_asyncio -q

import nest_asyncio, asyncio, os
from openai import AsyncOpenAI

# Allow async functions in Colab
nest_asyncio.apply()

# ------------------ Simple local replacements for agents ------------------
class Agent:
    def __init__(self, name, instructions, model):
        self.name = name
        self.instructions = instructions
        self.model = model

class OpenAIChatCompletionsModel:
    def __init__(self, model, openai_client):
        self.model = model
        self.client = openai_client

class Runner:
    @staticmethod
    def run_sync(agent, messages):
        """Run a conversation synchronously with memory."""
        async def _run():
            response = await agent.model.client.chat.completions.create(
                model=agent.model.model,
                messages=messages
            )
            return response.choices[0].message.content

        return asyncio.get_event_loop().run_until_complete(_run())

# ------------------ Gemini Setup ------------------
gemini_api_key = "key here"  # or load from env
client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

# ------------------ Agent Setup ------------------
agent = Agent(
    name="Assistant",
    instructions="You are an expert of agentic AI.",
    model=OpenAIChatCompletionsModel(
        model="gemini-2.0-flash",
        openai_client=client
    )
)

# ------------------ Conversation with Memory ------------------
history = [{"role": "system", "content": agent.instructions}]

while True:
    query = input("Enter the query (or 'exit' to stop): ")
    if query.lower() == "exit":
        break

    history.append({"role": "user", "content": query})

    result = Runner.run_sync(agent, history)

    history.append({"role": "assistant", "content": result})

    print("\n--- Assistant ---\n", result, "\n")


**Raw Response Event**

In [ ]:
!pip install openai nest_asyncio -q

import nest_asyncio, asyncio, os, time, sys
from openai import AsyncOpenAI

# Allow async inside Colab
nest_asyncio.apply()

# ------------------ Agent + Runner Classes ------------------
class Agent:
    def __init__(self, name, instructions, model):
        self.name = name
        self.instructions = instructions
        self.model = model

class OpenAIChatCompletionsModel:
    def __init__(self, model, openai_client):
        self.model = model
        self.client = openai_client

class Runner:
    @staticmethod
    def run_sync(agent, messages):
        """Runs a chat completion synchronously."""
        async def _run():
            response = await agent.model.client.chat.completions.create(
                model=agent.model.model,
                messages=messages
            )
            return response.choices[0].message.content

        return asyncio.get_event_loop().run_until_complete(_run())

# ------------------ Gemini Setup ------------------
gemini_api_key = "key here"  #  Replace with your API key
client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

# ------------------ Agent Setup ------------------
agent = Agent(
    name="Assistant",
    instructions="You are an expert AI assistant who explains concepts clearly.",
    model=OpenAIChatCompletionsModel(
        model="gemini-2.0-flash",
        openai_client=client
    )
)

# ------------------ Chat Loop ------------------
history = [{"role": "system", "content": agent.instructions}]

while True:
    query = input("Enter your question (or 'exit' to quit): ")
    if query.lower() == "exit":
        print("Goodbye! 👋")
        break

    history.append({"role": "user", "content": query})
    result = Runner.run_sync(agent, history)
    history.append({"role": "assistant", "content": result})

    # Fake typing effect for realism
    print("\n--- Assistant ---\n")
    for ch in result:
        print(ch, end="")
        sys.stdout.flush()
        time.sleep(0.01)
    print("\n")


**run_item_and_agent_events**

In [ ]:
!pip install openai nest_asyncio python-dotenv -q

import nest_asyncio, asyncio, os, sys, time, random
from openai import AsyncOpenAI

# Allow async in Colab
nest_asyncio.apply()

# ------------------ Custom lightweight framework ------------------

class ItemHelpers:
    @staticmethod
    def text_message_output(item):
        return item.get("content", "")

# ------------------ Custom Agent system ------------------
class Agent:
    def __init__(self, name, instructions, model, tools=None):
        self.name = name
        self.instructions = instructions
        self.model = model
        self.tools = tools or []

class OpenAIChatCompletionsModel:
    def __init__(self, model, openai_client):
        self.model = model
        self.client = openai_client

class Runner:
    @staticmethod
    async def run_streamed(agent, query):
        """Simulate streamed run events manually."""
        messages = [
            {"role": "system", "content": agent.instructions},
            {"role": "user", "content": query},
        ]

        # Fake stream behavior (since openai-python doesn’t expose granular events easily)
        print("=== Run starting ===")

        # 1️⃣ Detect if query involves a weather tool call
        if "weather" in query.lower():
            print("-- Tool was called: get_weather")
            output = agent.tools[0]("Lahore")  # simulate tool output
            await asyncio.sleep(1)
            print(f"-- Tool output: {output}")
            messages.append({"role": "assistant", "content": output})
        else:
            print("-- No tool call needed")

        # 2️⃣ Call model
        response = await agent.model.client.chat.completions.create(
            model=agent.model.model,
            messages=messages
        )
        text = response.choices[0].message.content
        print("-- Message output:")
        print(text)

        print("=== Run complete ===")
        return text

# ------------------ Gemini Setup ------------------
gemini_api_key = "YOUR_GEMINI_API_KEY"  # 🔑 Replace with your real API key
client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

# ------------------ Define Tool ------------------
def get_weather(city: str) -> str:
    # Just a dummy weather function
    conditions = ["sunny ☀️", "rainy 🌧️", "cloudy ☁️", "windy 🌬️"]
    return f"The weather in {city} is {random.choice(conditions)}."

# ------------------ Agent Setup ------------------
agent = Agent(
    name="Assistant",
    instructions="You are a friendly weather assistant.",
    model=OpenAIChatCompletionsModel(
        model="gemini-2.0-flash",
        openai_client=client
    ),
    tools=[get_weather]
)

# ------------------ Chat Loop ------------------
while True:
    query = input("Enter your question (or 'exit' to quit): ")
    if query.lower() == "exit":
        print("Goodbye! 👋")
        break

    # Run streamed version (async)
    asyncio.get_event_loop().run_until_complete(Runner.run_streamed(agent, query))
    print("\n")


# Tool

In [ ]:
# --- 🧩 Example: Weather Fetch Tool with Async Streaming Output ---

!pip install openai pydantic asyncio -q

import asyncio
from pydantic import BaseModel
from openai import AsyncOpenAI
from openai.types.responses import ResponseTextDeltaEvent

# --- Initialize client ---
client = AsyncOpenAI(api_key="YOUR_OPENAI_API_KEY")

# --- Dummy RunContextWrapper class ---
class RunContextWrapper:
    def __init__(self, data=None):
        self.data = data

# --- Simulated weather fetching function ---
def fetch_weather_data(city: str, unit: str) -> str:
    if city.lower() == "lahore":
        temp = 30 if unit == "C" else 86
    elif city.lower() == "karachi":
        temp = 33 if unit == "C" else 91
    else:
        temp = 25 if unit == "C" else 77
    return f"The current temperature in {city} is {temp}°{unit}."

# --- Input model for the tool ---
class WeatherArgs(BaseModel):
    city: str
    unit: str

# --- Fixed function (removed [Any]) ---
async def run_fetch_weather(ctx: RunContextWrapper, args: str) -> str:
    parsed = WeatherArgs.model_validate_json(args)
    return fetch_weather_data(city=parsed.city, unit=parsed.unit)

# --- Async main function ---
async def main():
    print("Type a city name (e.g., Lahore, Karachi, or Islamabad):")
    city = input("City: ")
    unit = input("Unit (C/F): ")

    args = WeatherArgs(city=city, unit=unit).model_dump_json()
    ctx = RunContextWrapper()

    print("\n🤖 Fetching weather info...\n")
    result_text = await run_fetch_weather(ctx, args)

    # Simulate streaming output like ChatGPT
    for ch in result_text:
        print(ch, end="", flush=True)
        await asyncio.sleep(0.03)
    print("\n\n✅ Done!")

# --- Run the async main loop ---
await main()


**Function Tool**

In [ ]:
!pip install -q openai python-dotenv nest_asyncio

import os
import nest_asyncio
import asyncio
from dotenv import load_dotenv
from openai import AsyncOpenAI

# Allow async code inside Colab
nest_asyncio.apply()

# --- Load API key ---
load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY")

if not gemini_api_key:
    gemini_api_key = input("Enter your Gemini API key: ")

# --- Create client ---
client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

# --- Define fake tools ---
async def fetch_weather(location: str) -> str:
    return f"The weather in {location} is 28°C and sunny."

def fetch_news(location: str) -> str:
    return f"Breaking news from {location}: AI revolution continues!"

def fetch_stock_price(company: str) -> str:
    return f"The current stock price of {company} is USD 1000.00."

# --- Main chat function ---
async def main():
    print("🌦 Welcome to the Gemini Assistant!")
    print("Ask about weather, news, or stock prices.\n")

    while True:
        query = input("Enter your query (or 'exit' to quit): ")
        if query.lower() == "exit":
            print("Goodbye 👋")
            break

        if "weather" in query.lower():
            location = query.split("in")[-1].strip() if "in" in query.lower() else "your area"
            response = await fetch_weather(location)
        elif "news" in query.lower():
            location = query.split("in")[-1].strip() if "in" in query.lower() else "your area"
            response = fetch_news(location)
        elif "stock" in query.lower():
            company = query.split("for")[-1].strip() if "for" in query.lower() else "Tesla"
            response = fetch_stock_price(company)
        else:
            completion = await client.chat.completions.create(
                model="gemini-2.0-flash",
                messages=[{"role": "user", "content": query}],
            )
            response = completion.choices[0].message.content

        print(f"\n🤖 {response}\n")

# --- Run async main() safely in Colab ---
await main()


**Hosted Tool**

In [ ]:
!pip install openai nest_asyncio python-dotenv -q

import nest_asyncio, asyncio, os, time, sys
from openai import AsyncOpenAI

# Allow async inside Colab
nest_asyncio.apply()

# ------------------ Basic Agent Classes ------------------
class Agent:
    def __init__(self, name, instructions, model, tools=None):
        self.name = name
        self.instructions = instructions
        self.model = model
        self.tools = tools or []

class OpenAIChatCompletionsModel:
    def __init__(self, model, openai_client):
        self.model = model
        self.client = openai_client

class Runner:
    @staticmethod
    def run_sync(agent, query):
        """Runs a chat completion synchronously."""
        async def _run():
            messages = [
                {"role": "system", "content": agent.instructions},
                {"role": "user", "content": query}
            ]
            response = await agent.model.client.chat.completions.create(
                model=agent.model.model,
                messages=messages
            )
            return response.choices[0].message.content
        return asyncio.get_event_loop().run_until_complete(_run())

# ------------------ Tool Simulations ------------------
class WebSearchTool:
    def __init__(self):
        pass
    def search(self, query):
        # Simulated search tool
        return f"(Simulated) Web search results for: '{query}'"

class FileSearchTool:
    def __init__(self, max_num_results=3, vector_store_ids=None):
        self.max_num_results = max_num_results
        self.vector_store_ids = vector_store_ids or []
    def search(self, query):
        # Simulated file search
        return f"(Simulated) File search for '{query}' in {self.vector_store_ids}"

# ------------------ Gemini Setup ------------------
gemini_api_key = input("Enter your Gemini API key: ").strip()
client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

# ------------------ Agent Setup ------------------
agent = Agent(
    name="Assistant",
    instructions="You are an expert of agentic AI.",
    model=OpenAIChatCompletionsModel(
        model="gemini-2.0-flash",
        openai_client=client
    ),
    tools=[
        WebSearchTool(),
        FileSearchTool(max_num_results=3, vector_store_ids=["VECTOR_STORE_ID"]),
    ]
)

# ------------------ Run Agent ------------------
while True:
    query = input("\nEnter your query (or 'exit' to quit): ")
    if query.lower() == "exit":
        print("Goodbye! 👋")
        break

    # Simulate using tools
    print("\n🔍 Tool outputs:")
    for tool in agent.tools:
        if hasattr(tool, "search"):
            print(" -", tool.search(query))

    # Get AI response
    print("\n🤖 Assistant response:")
    result = Runner.run_sync(agent, query)
    for ch in result:
        print(ch, end="")
        sys.stdout.flush()
        time.sleep(0.01)
    print("\n")


In [ ]:
# @title
!pip install openai python-dotenv --quiet

import asyncio
from openai import AsyncOpenAI
import os
from dotenv import load_dotenv

# --- Set up your Gemini API key ---
GEMINI_API_KEY = input("Enter your Gemini API key: ")

# --- Create a simple Agent system manually ---
class Agent:
    def __init__(self, name, instructions):
        self.name = name
        self.instructions = instructions

    def as_tool(self, tool_name, tool_description):
        return {"tool_name": tool_name, "tool_description": tool_description, "agent": self}

    async def run(self, input_text):
        if "spanish" in self.instructions.lower():
            return f"Traducción al español: '{input_text}' → 'Hola, ¿cómo estás?'"
        elif "french" in self.instructions.lower():
            return f"Traduction en français: '{input_text}' → 'Bonjour, comment ça va ?'"
        else:
            return f"{self.name} says: {input_text}"

class Runner:
    @staticmethod
    async def run(agent, input):
        return await agent.run(input)

# --- Create the agents ---
spanish_agent = Agent(
    name="Spanish agent",
    instructions="You translate the user's message to Spanish",
)

french_agent = Agent(
    name="French agent",
    instructions="You translate the user's message to French",
)

orchestrator_agent = Agent(
    name="orchestrator_agent",
    instructions=(
        "You are a translation agent. You use the tools given to you to translate."
        "If asked for multiple translations, you call the relevant tools."
    ),
)

# --- Simulation: orchestrator chooses the Spanish agent ---
async def main():
    result = await spanish_agent.run("Say 'Hello, how are you?' in french.")
    print(result)

# --- Run in Colab safely ---
await main()


**Conditional tool enabling**

In [ ]:
!pip install nest_asyncio pydantic -q
import nest_asyncio, asyncio
from pydantic import BaseModel

nest_asyncio.apply()

# -----------------------
#  Simulate Agent Classes
# -----------------------

class RunContextWrapper:
    def __init__(self, context):
        self.context = context

class AgentBase:
    pass

class Agent(AgentBase):
    def __init__(self, name, instructions):
        self.name = name
        self.instructions = instructions
        self.tools = []

    def as_tool(self, tool_name, tool_description, is_enabled=True):
        return {
            "tool_name": tool_name,
            "tool_description": tool_description,
            "is_enabled": is_enabled,
            "agent": self
        }

class Runner:
    @staticmethod
    async def run(agent, input_text, context=None):
        enabled_tools = []
        for tool in agent.tools:
            is_enabled = tool["is_enabled"]
            if callable(is_enabled):
                enabled = is_enabled(RunContextWrapper(context), agent)
            else:
                enabled = is_enabled
            if enabled:
                enabled_tools.append(tool)

        responses = []
        for tool in enabled_tools:
            responses.append(f"{tool['agent'].name} → '{tool['agent'].instructions}' → Responding to: {input_text}")

        return type("Result", (), {"final_output": "\n".join(responses)})

# -----------------------
#  Define Context & Logic
# -----------------------

class LanguageContext(BaseModel):
    language_preference: str = "french_spanish"

def french_enabled(ctx: RunContextWrapper, agent: AgentBase) -> bool:
    return ctx.context.language_preference == "french_spanish"

# Specialized agents
spanish_agent = Agent(
    name="spanish_agent",
    instructions="You respond in Spanish. Always reply to the user's question in Spanish."
)

french_agent = Agent(
    name="french_agent",
    instructions="You respond in French. Always reply to the user's question in French."
)

# Orchestrator agent
orchestrator = Agent(
    name="orchestrator",
    instructions=(
        "You are a multilingual assistant. You use tools to respond in different languages."
    )
)
orchestrator.tools = [
    spanish_agent.as_tool(
        tool_name="respond_spanish",
        tool_description="Respond to the user's question in Spanish",
        is_enabled=True,
    ),
    french_agent.as_tool(
        tool_name="respond_french",
        tool_description="Respond to the user's question in French",
        is_enabled=french_enabled,
    ),
]

# -----------------------
#  Run in Colab
# -----------------------

async def main():
    context = LanguageContext(language_preference="french_spanish")
    result = await Runner.run(orchestrator, "How are you?", context=context)
    print("=== FINAL OUTPUT ===")
    print(result.final_output)

await main()
